In [1]:
%serialconnect

Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [2]:
%sendtofile --source ssd1306.py


Sent 168 lines (5477 bytes) to ssd1306.py.


In [2]:
#https://forum.micropython.org/viewtopic.php?f=18&p=23080
from machine import I2C, Pin
import ssd1306

rst = Pin(16, Pin.OUT)
rst.value(1)
scl = Pin(15, Pin.OUT, Pin.PULL_UP)
sda = Pin(4, Pin.OUT, Pin.PULL_UP)
i2c = I2C(scl=scl, sda=sda, freq=450000)
o = ssd1306.SSD1306_I2C(128, 64, i2c, addr=0x3c)

o.fill(0)
o.text('ESP32', 45, 5)
o.text('Micro Alex Python', 20, 20)
o.show() 

In [7]:
help(o.framebuf)

object <FrameBuffer> is of type FrameBuffer
  fill -- <function>
  fill_rect -- <function>
  pixel -- <function>
  hline -- <function>
  vline -- <function>
  rect -- <function>
  line -- <function>
  blit -- <function>
  scroll -- <function>
  text -- <function>


In [13]:
import network
si = network.WLAN(network.STA_IF)


W (569481) phy_init: failed to load RF calibration data (0x1102), falling back to full calibration


In [17]:
si.active(True)
help(si)
x = si.scan()

object <WLAN> is of type WLAN
  active -- <function>
  connect -- <function>
  disconnect -- <function>
  status -- <function>
  scan -- <function>
  isconnected -- <function>
  config -- <function>
  ifconfig -- <function>


[(b'The Sensible Code Company', b'\xe4\xf4\xc6\x1dl\xf2', 12, -55, 3, False), (b'BluPrintWIFI', b'\x00\x1d\xaa\x19\xf6\xa0', 11, -65, 4, False), (b'BluPrint-GuestWifi', b'\x02\x1d\xaa\x18\xf6\xa0', 11, -67, 0, False), (b'DIRECT-vMC1860 Series', b'2\xcd\xa7*h\xbf', 11, -68, 3, False), (b'LSP Guest WIFI ', b'\x00\x04\x96\x89s\xc1', 6, -74, 4, False), (b'WiFi Secure', b'\x00\x04\x96\x89s\xc4', 6, -75, 5, False), (b'LSP Studio WiFi', b'\x00\x04\x96\x89s\xc0', 6, -75, 4, False), (b'WiFi MAC', b'\x00\x04\x96\x89s\xc5', 6, -75, 0, False), (b'LJMUsetup', b'\xf4\x1f\xc2\xe7\x80\x00', 11, -81, 0, False), (b'rcrain-timemachine', b'lp\x9f\xdd_\x84', 6, -82, 3, False), (b'mirror', b'\xf4\x1f\xc2\xe7\x80\x06', 11, -82, 3, False), (b'IoT', b'\xf4\x1f\xc2\xe7\x80\x05', 11, -82, 4, False), (b'loanlap', b'\xf4\x1f\xc2\xe7\x80\x04', 11, -82, 0, False), (b'scan', b'\xf4\x1f\xc2\xe7\x80\x02', 11, -82, 4, False), (b'eduroam', b'\xf4\x1f\xc2\xe7\x80\x01', 11, -82, 5, False), (b'LJMU_Guest', b'\xf4\x1f\xc2\xe

In [51]:
for w in x:
    print(w[3], w[0])


-55 b'The Sensible Code Company'
-65 b'BluPrintWIFI'
-67 b'BluPrint-GuestWifi'
-68 b'DIRECT-vMC1860 Series'
-74 b'LSP Guest WIFI '
-75 b'WiFi Secure'
-75 b'LSP Studio WiFi'
-75 b'WiFi MAC'
-81 b'LJMUsetup'
-82 b'rcrain-timemachine'
-82 b'mirror'
-82 b'IoT'
-82 b'loanlap'
-82 b'scan'
-82 b'eduroam'
-82 b'LJMU_Guest'
-82 b'legacy'
-83 b'TP-LINK_96CE38'
-84 b'\xef\xa3\xbf AMRS'
-85 b'Aspirate n Go'
-86 b'LJMU_Guest'
-86 b'IoT'
-87 b'legacy'


In [57]:
ww = [ ]
for w in x:
    ww.append(w[0])

print(ww)

[b'The Sensible Code Company', b'BluPrintWIFI', b'BluPrint-GuestWifi', b'DIRECT-vMC1860 Series', b'LSP Guest WIFI ', b'WiFi Secure', b'LSP Studio WiFi', b'WiFi MAC', b'LJMUsetup', b'rcrain-timemachine', b'mirror', b'IoT', b'loanlap', b'scan', b'eduroam', b'LJMU_Guest', b'legacy', b'TP-LINK_96CE38', b'\xef\xa3\xbf AMRS', b'Aspirate n Go', b'LJMU_Guest', b'IoT', b'legacy']


In [63]:
def ll(w):
    return w[-1]

ww.sort(key=ll)
print(ww)

[b'LSP Guest WIFI ', b'TP-LINK_96CE38', b'WiFi MAC', b'BluPrintWIFI', b'\xef\xa3\xbf AMRS', b'IoT', b'IoT', b'rcrain-timemachine', b'WiFi Secure', b'BluPrint-GuestWifi', b'LSP Studio WiFi', b'eduroam', b'scan', b'Aspirate n Go', b'LJMUsetup', b'loanlap', b'mirror', b'DIRECT-vMC1860 Series', b'LJMU_Guest', b'LJMU_Guest', b'The Sensible Code Company', b'legacy', b'legacy']


In [65]:
print("A"<"a")

True


In [38]:
b = { "sss":9, 777:3, "aaa":"bbb"}
b[(9,8)] = 999
print(b)


{'aaa': 'bbb', 777: 3, 'sss': 9, (9, 8): 999}


In [45]:
v = (10,11)
b[v] = 666
print(b)

{'sss': 9, (10, 11): 666, 777: 3, 'aaa': 'bbb', (9, 8): 999}


In [30]:
a = []

('aaa', 7, [99, 'ooo', ('aaa', 7, [99, 'ooo', ('aaa', 7, [99, 'ooo', ('aaa', 7, [99, 'ooo', ('aaa', 7, [99, 'ooo', ('aaa', 7, [99, 'ooo', ('aaa', 7, [99, 'ooo', ('aaa', 7, [99, 'ooo', ('aaa', 7, [99, 'ooo', ('aaa', 7, [99, 'ooo', ('aaa', 7, [99, 'ooo', ('aaa', 7, [99, 'ooo', ('aaa', 7, [99, 'ooo', ('aaa', 7, [99, 'ooo', ('aaa', 7, [99, 'ooo', ('aaa', 7, [99, 'ooo', ('aaa', 7, [99, 'ooo', ('aaa', 7, [99, 'ooo', ('aaa', 7, [99, 'ooo', ('aaa', 7, [99, 'ooo', ('aaa', 7, [99, 'ooo', ('aaa', 7, [99, 'ooo', ('aaa', 7, [99, 'ooo', ('aaa', 7, [99, 'ooo', ('aaa', 7, [99, 'ooo', ('aaa', 7, [99, 'ooo', ('aaa', 7, [99, 'ooo', ('aaa', 7, [99, 'ooo', ('aaa', 7, [99, 'ooo', ('aaa', 7, [99, 'ooo', ('aaa', 7, [99, 'ooo', ('aaa', 7, [99, 'ooo', ('aaa', 7, [99, 'ooo', ('aaa', 7, [99, 'ooo', ('aaa', 7, [99, 'ooo', ('aaa', 7, [99, 'ooo', ('aaa', 7, [99, 'ooo', ('aaa', 7, [99, 'ooo', ('aaa', 7, [99, 'ooo', ('aaa', 7, [99, 'ooo', ('aaa', 7, [99, 'ooo', ('aaa', 7, [99, 'ooo', ('aaa', 7, [99, 'ooo', ('aaa', 7, 

Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
RuntimeError: maximum recursion depth exceeded


In [11]:
import network
from syslog import log

si = network.WLAN(network.STA_IF)
macaddress = "".join("{:02x}".format(x)  for x in si.config("mac"))
si.active(True)



In [27]:
o.fill(0)
for i in range(128):
    o.framebuf.fill_rect(1, 3, i, 34, 1)
    o.show()

In [28]:
%disconnect

attempt to exit paste mode
[\r\x03\x02] b'\r\nMicroPython v1.9.3-454-g34e224a4 on 2018-03-14; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> '
Closing serial Serial<id=0x7fc884a66898, open=True>(port='/dev/ttyUSB0', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)
